In [1]:
import pandas as pd
from Levenshtein import distance

In [2]:
from sqlalchemy import create_engine

In [3]:
conn = create_engine('mariadb+mariadbconnector://root@mariadb:3306/sh')

## Sources for SortingHat Identities

In [4]:
pd.read_sql("SELECT distinct(source) FROM identities", conn)

,source
0,github
1,github2
2,gitter
3,git


# Unmapped Gitter identities

In [5]:
# get merged gitter to github identities
mapped_gitter = pd.read_sql("""
            select distinct(i1.uuid) from identities i1
            left join (select * from identities where source='github') i2 on i2.uuid = i1.uuid
            left join (select * from identities where source='github2') i3 on i3.uuid = i2.uuid
            where i1.source='gitter'
            and (i2.uuid is not null or i3.uuid is not null)
            """, conn)['uuid'].values

In [6]:
# identities that have not been mapped to gitter
unmapped_gitter = pd.read_sql("""
            select * from identities where source='gitter'
            and uuid not in {}
            """.format(str(tuple(mapped_gitter))), conn)

# Unmapped Github identities

In [7]:
unmapped_github = pd.read_sql("""
            SELECT * FROM identities where source in ('github', 'github2') and id=uuid
            and uuid not in {}
            """.format(str(tuple(mapped_gitter))), conn)

# Attempt to map unmapped Github and Gitter identities

Using Levenshtein distance metric we attempt to match users using combined name and username similarity

Note that this a naive implementation and could be possibly improved

In [8]:
potential_matches = []

for i1, row1 in unmapped_github[~unmapped_github['uuid'].isin(unmapped_gitter['uuid'].unique())].iterrows():
    for i2, row2 in unmapped_gitter[~unmapped_gitter['uuid'].isin(unmapped_github['uuid'].unique())].iterrows():
        name_similarity = 1 - (distance(str(row1['name']).lower(), str(row2['name']).lower()) / max(len(str(row1['name'])), len(str(row2['name']))))
        username_similarity = 1 - (distance(str(row1['username']).lower(), str(row2['username']).lower()) / max(len(str(row1['username'])), len(str(row2['username']))))
        if name_similarity > 0.7 and username_similarity > 0.7 and row1['uuid'] != row2['uuid']:
            potential_matches.append({
                'name_similarity': name_similarity,
                'username_similarity': username_similarity,
                'github_row': row1,
                'gitter_row': row2
            })
            
            print(i1, i2)

100 3343
754 1189
811 835
1187 1394
2070 149
3269 1098
3510 305
3555 1639
3645 1708
3669 795
4049 2184
4791 3586


In [9]:
for item in potential_matches:
    print("'{}' and '{}' have {} name similarity".format(item['github_row']['name'], item['gitter_row']['name'], item['name_similarity']))
    print("'{}' and '{}' have {} username similarity".format(item['github_row']['username'], item['gitter_row']['username'], item['username_similarity']))
    print("Go to {} and search for name '{}' / username '{}' if similar enough\n\n".format('http://localhost:8000/identities/hatstall/{}/'.format(item['gitter_row']['uuid']), item['github_row']['name'], item['github_row']['username']))

'Brandon Cole' and 'Brandon Couts' have 0.7692307692307692 name similarity
'brandoncole' and 'BrandonCouts' have 0.75 username similarity
Go to http://localhost:8000/identities/hatstall/edd71d58fa8b3182c5e0aa0ea33b6e6ed8c5773a/ and search for name 'Brandon Cole' / username 'brandoncole' if similar enough


'Adrien Grondin' and 'AdrienGrondin' have 0.9285714285714286 name similarity
'adrgrondin' and 'AdrienGrondin' have 0.7692307692307692 username similarity
Go to http://localhost:8000/identities/hatstall/5470147aea5b00b1cf1dc997375f63a87b5fc6c8/ and search for name 'Adrien Grondin' / username 'adrgrondin' if similar enough


'Kris Shinn' and 'Eric Shinn' have 0.8 name similarity
'kshinn' and 'eshinn' have 0.8333333333333334 username similarity
Go to http://localhost:8000/identities/hatstall/3b9dcb5c6ff10f18bb02875dd40a89bc0fd29dcb/ and search for name 'Kris Shinn' / username 'kshinn' if similar enough


'Andrew Revinsky' and 'Andrew Revinsky' have 1.0 name similarity
'andrevinsky' and 